In [1]:
import numpy as np
import pandas as pd

things_filename = "./data/sourceThings.csv"
modifiers_filename = "./data/modifiers.csv"

things=pd.read_csv(things_filename)
modifiers=pd.read_csv(modifiers_filename)

things.head()

,name,size,can_fly,good,danger,speed
0,ape,3,-1,0,1,4
1,banshee,2,0,-1,1,6
2,basilisk,3,-1,-1,1,4
3,bass,2,-1,0,-1,3
4,bear,4,-1,0,1,5


In [2]:
def resolve_categorical_override(base_value, override1, override2):
    # If both overrides agree on a non-neutral value, use it
    if override1 == override2 and override1 != 0:
        return override1

    # If one override is neutral and the other is not, use the non-neutral one
    if override1 == 0:
        return override2
    if override2 == 0:
        return override1

    # If overrides are conflicting or both neutral, use the base value
    return base_value

In [3]:
import csv
import random
all_data_filename = "./data/superData.csv"

with open(all_data_filename, 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    
    # Write the header row
    header = ['name', 'unmodified_size', 'modified_size', 'can_fly', 'good', 'danger', 'modified_speed', 'combat_value']
    writer.writerow(header)
    
    things = things.reset_index()  # make sure indexes pair with number of rows
    modifiers = modifiers.reset_index()  # make sure indexes pair with number of rows

    # Assuming the modifiers dataframe is already loaded and updated
    for sindex, thing in things.iterrows():
        for mindex, modifier in modifiers.iterrows():
            # Apply size modifier
            unmodified_size = thing['size']
            modified_size = thing['size'] + modifier['size']
            modified_speed = thing['speed'] + modifier['speed']
    
            # Apply categorical overrides
            can_fly = modifier['can_fly_override'] if modifier['can_fly_override'] != 0 else thing['can_fly']
            good = modifier['good_override'] if modifier['good_override'] != 0 else thing['good']

            combat_value = (modified_size + modified_speed) * modifier['combat_value_adjustment']
            
            # Determine danger based on size
            if (modified_size >= 5)  or (combat_value >= 10):
                danger = 1
            elif (modified_size <= 1) or (combat_value <= 3):
                danger = -1
            else:
                danger = modifier['danger_override'] if modifier['danger_override'] != 0 else thing['danger']


            # Combine data
            data = [modifier['modifier'] + ' ' + thing['name'], 
                    unmodified_size,
                    modified_size,
                    can_fly,
                    good,
                    danger,
                    modified_speed,
                    combat_value
                   ]
            writer.writerow(data)
            
    for _ in range(20000):
        thing = random.choice(things.to_dict(orient='records'))
        modifier1, modifier2 = random.sample(modifiers.to_dict(orient='records'), 2)

        # Apply size and speed adjustments additively
        unmodified_size = thing['size']
        modified_size = thing['size'] + modifier1['size'] + modifier2['size']
        modified_speed = thing['speed'] + modifier1['speed'] + modifier2['speed']

        # Calculate combat value for each modifier
        combat_value = ((modified_size + modified_speed) * modifier1['combat_value_adjustment'] * modifier2['combat_value_adjustment'])

        # Apply categorical overrides
        can_fly = resolve_categorical_override(thing['can_fly'], modifier1['can_fly_override'], modifier2['can_fly_override'])
        good = resolve_categorical_override(thing['good'], modifier1['good_override'], modifier2['good_override'])

        # Determine danger based on size
        if (modified_size >= 5)  or (combat_value >= 10):
            danger = 1
        elif (modified_size <= 1) or (combat_value <= 3):
            danger = -1
        else:
            danger = modifier['danger_override'] if modifier['danger_override'] != 0 else thing['danger']


        # Combine data and write to file
        data = [f"{modifier1['modifier']} {modifier2['modifier']} {thing['name']}", unmodified_size, modified_size, can_fly, good, danger, modified_speed, combat_value]
        writer.writerow(data)